In [1]:
#import depedencies
import pandas as pd
import fiona
import folium
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import FastMarkerCluster,HeatMap

In [2]:
#import the data
df_restaurants = pd.read_csv("../data/restaurants_counts.csv")
df_diebeties = pd.read_csv("../data/dibeties_2019data.csv")
df_restaurants.head()

,Unnamed: 0,State,counts,State.1,Latitude,Longitude
0,0,AK,14,AK,61.302501,-158.775020
1,1,AL,236,AL,32.601011,-86.680736
2,2,AR,151,AR,34.751928,-92.131378
3,3,AZ,208,AZ,34.168219,-111.930907
4,4,CA,676,CA,37.271875,-119.270415


In [3]:
df_2 = df_diebeties.dropna()
df_2.head()

,Unnamed: 0,Unnamed: 0.1,yearstart,locationabbr,locationdesc,question,datavalueunit,datavaluetype,datavalue,State,Latitude,Longitude
0,5,5930,2019,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.1,AK,61.302501,-158.775020
1,11,5938,2019,AL,Alabama,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,12.2,AL,32.601011,-86.680736
2,17,6018,2019,AR,Arkansas,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,12.2,AR,34.751928,-92.131378
3,23,6199,2019,AZ,Arizona,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,9.8,AZ,34.168219,-111.930907
4,29,6291,2019,CA,California,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,9.4,CA,37.271875,-119.270415


In [4]:
#creating the base map
m= folium.Map(location = [38.27312, -98.5821872], zoom_start = 4)

#creating bubble map with restaurant counts per state.
#Loop through each row in the dataframe
for i in range(0,len(df_restaurants)):
   folium.Circle(
      location=[df_restaurants.iloc[i]['Latitude'], df_restaurants.iloc[i]['Longitude']],
      popup=df_restaurants.iloc[i]['counts'],
      radius=float(df_restaurants.iloc[i]['counts'])*200,
      color='crimson',
      fill=True,
      fill_color='crimson'
      ).add_to(m)
    
m

In [5]:
# Add a HeatMap map to the base map for Dibeties per 1000
lats_longs_weight = list(map(list, zip(df_2["Latitude"],
                          df_2["Longitude"],
                          df_2["datavalue"]
                         )
               )
           )

#Creating a title for the layers in map
fg = folium.FeatureGroup(name="Diabeties HeatMap")
fg.add_child(HeatMap(lats_longs_weight))
m.add_child(fg)
m


In [6]:
#Creating Statenames and tooltip with Datavalue showing Diabeties per 1000
for i in range(0,len(df_2)):
   folium.Marker(
      location=[df_2.iloc[i]['Latitude'], df_2.iloc[i]['Longitude']],
      tooltip=df_2.iloc[i]['datavalue'],
      icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{df_2.iloc[i]['locationdesc']}</div>""")
      ).add_to(m)
folium.LayerControl().add_to(m)
m

In [8]:
#Saving map file
# m.save("Restaurants_vs_Diabeties_per_1000_map.html")